In [1]:
import re
# import Python packages as needed
# NOTE: you can choose to install/use external packages

# Question 1: word association mining

## Basic statistics of the corpus

In [2]:
review_filepath = './amazon_reviews.txt'

In [3]:
num_reviews = 0
num_positive_reviews = 0
num_negative_reviews = 0
with open(review_filepath) as f:
    f.readline() # skip header (the first line) 
    for line in f:
        num_reviews += 1
        if line.strip().split()[0] == '1':
            num_positive_reviews += 1
        else:
            num_negative_reviews += 1
print ('total number of reviews:', num_reviews)
print ('total number of positive reviews:', num_positive_reviews)
print ('total number of negative reviews:', num_negative_reviews)

total number of reviews: 30000
total number of positive reviews: 15091
total number of negative reviews: 14909


## Count frequency of single words

In [4]:
def process_text(text):
    for punctuations in [',', '.', '"', '!', '?', ':', ';', '-', '(', ')', '[', ']']:
        text = text.replace(punctuations, ' ')
    text = re.sub('\s+', ' ', text)
    text = text.lower().strip()
    return text

In [5]:
# Count the frequency of single words (aka. unigrams) in the corpus
# Parameter:
#       filepath: file path of amazon_review.txt
# Return: 
#       a dictionary, key = word, value = word frequency

def get_single_word_frequency(filepath):
    word_freq = {}
    with open(filepath) as f:
        f.readline() # skip header (the first line) 
        for line in f:
            review_text = process_text(line.split('\t')[1])
            for word in review_text.split():
                if word not in word_freq:
                    word_freq[word] = 1
                else:
                    word_freq[word] += 1
    return word_freq

In [6]:
word_freq = get_single_word_frequency(review_filepath)
for word, freq in sorted(word_freq.items(), key = lambda x: x[1], reverse = True)[:10]:
    print(word, freq)

the 119835
and 64619
i 63045
a 60750
to 57968
it 47813
of 47382
this 44363
is 41185
in 27962


In [7]:
total_num_words = sum(word_freq.values())
print ('number of unique words:', len(word_freq))
print ('total number of word occurrences:', total_num_words)

number of unique words: 69034
total number of word occurrences: 2384092


## Count frequency of ordered pair of words in a text window

In [8]:
# Count the number of text windows that contain an ordered pair of words
# Parameter:
#       filepath: file path of amazon_review.txt
#       window_size: the size of a text window (measured in number of words)
# Return: 
#       a dictionary, key = ordered word pair (a tuple), 
#                     value = number of text windows containing this pair

def get_ordered_word_pair_frequency(filepath, window_size):
    pair_freq = {}
    with open(filepath) as f:
        f.readline() # skip header (the first line) 
        for line in f:
            review_text = process_text(line.split('\t')[1])
            word_list = review_text.split()
            for i in range(len(word_list)):
                for j in range(i + 1, len(word_list)):
                    # only consider pairs of words no more than window_size apart  
                    if j - i + 1 >= window_size:
                        break
                    # put this ordered word pair into a tuple
                    order_word_pair = (word_list[i], word_list[j])
                    # accumulate counts
                    if order_word_pair not in pair_freq:
                        pair_freq[order_word_pair] = 1
                    else:
                        pair_freq[order_word_pair] += 1
    return pair_freq

In [9]:
TEXT_WINDOW_SIZE = 5
pair_freq = get_ordered_word_pair_frequency(review_filepath, TEXT_WINDOW_SIZE)
for pair, freq in sorted(pair_freq.items(), key = lambda x: x[1], reverse = True)[:10]:
    print(pair, freq)

('of', 'the') 15208
('the', 'of') 12728
('to', 'the') 11803
('this', 'is') 10832
('the', 'the') 10615
('and', 'the') 10479
('in', 'the') 9092
('the', 'and') 8728
('the', 'is') 8420
('is', 'a') 8106


## Calculate pointwise mutual information for each ordered pair

In [10]:
WORD_PAIR_FREQUENCY_THRESHOLD = 50
pmi_per_pair = {}
for pair, freq in pair_freq.items():
    if freq < WORD_PAIR_FREQUENCY_THRESHOLD: # filter out infrequent word pairs
        continue
    if pair[0] in word_freq and pair[1] in word_freq:
        # calculate the pointwise mutual information for this pair of words
        # you may or may not need to use the following variables:
        # 
        # freq: frequency of this word pair
        # word_freq[pair[0]]: frequency of the first word in the pair
        # word_freq[pair[1]]: frequency of the second word in the pair
        # total_num_words: total number of words in the corpus (i.e. corpus size)
        # 
        # pmi_per_pair[pair] = 
        
        continue # once you fill out the code, you can comment out this line

In [11]:
# sort word pairs in pmi_per_pair by their PMI from highest to lowest. Show the top 100 pairs.

# Question 2: feature selection using Chi-square statistic

## For each word, count how many positive (negative) documents it appears in

In [12]:
# Count the number of documents that has a specified sentiment and contain a single word  
# Parameter:
#       filepath: file path of amazon_review.txt
#       label: string '0' (negative) or '1' (positive).   
# Return: 
#       a dictionary, key = word, value = word frequency

def get_single_word_doc_frequency_per_label(filepath, label):
    word_freq_per_label = {}
    with open(filepath) as f:
        f.readline() # skip header (the first line) 
        for line in f:
            sentiment_label = line.split('\t')[0].strip()
            if sentiment_label == label:
                review_text = process_text(line.split('\t')[1])
                for word in set(review_text.split()):
                    if word not in word_freq_per_label:
                        word_freq_per_label[word] = 1
                    else:
                        word_freq_per_label[word] += 1
    return word_freq_per_label

In [13]:
# number of positive documents that contain a word 
positive_word_freq = get_single_word_doc_frequency_per_label(review_filepath, '1')
for word, freq in sorted(positive_word_freq.items(), key = lambda x: x[1], reverse = True)[:10]:
    print(word, freq)

the 13245
and 12526
a 11891
to 11156
this 11082
i 10334
is 10106
of 9998
it 9793
in 8182


In [14]:
# number of negative documents that contain a word 
negative_word_freq = get_single_word_doc_frequency_per_label(review_filepath, '0')
for word, freq in sorted(negative_word_freq.items(), key = lambda x: x[1], reverse = True)[:10]:
    print(word, freq)

the 13615
and 11743
a 11675
to 11463
i 11415
this 11379
it 10358
of 10128
is 9420
not 8236


## Calculate Chi-square statistic for each word

In [15]:

# contingency table per word:
#                                             sentiment
#                       positive                            negative
#               ------------------------------------------------------------------------
#       present | word present, positive sentiment | word present, negative sentiment |  
# word          ------------------------------------------------------------------------
#       absent  | word absent,  positive sentiment | word absent, negative sentiment  |  
#               ------------------------------------------------------------------------
#      

chi2_per_word = {}
for word, freq in word_freq.items():
#     if freq < 10: # filter infrequent words
#         continue
    if word in positive_word_freq and word in negative_word_freq:        
        # calculate the Chi-square statistic for this word
        # you may or may not need to use the following variables:
        # 
        # positive_word_freq[word]: number of positive reviews where this word is present
        # negative_word_freq[word]: number of negative reviews where this word is present 
        # num_positive_reviews: number of positive reviews in total 
        # num_negative_reviews: number of negative reviews in total
        # num_reviews: total number of reviews in the corpus
        # 
        # chi2_per_word[word] = 
        
        continue # once you fill out the code, you can comment out this line
        

In [16]:
# sort words in chi2_per_word by their Chi-square value from highest to lowest. Show the top 100 words.

# Question 3: spell correction using letter n-grams

In [17]:
a_list_filepath = './enwiktionary.a.list'

In [18]:
a_list = []
with open(a_list_filepath) as f:
    for line in f:
        a_list.append(line.strip())

In [19]:
print ('number of words/phrases in the list:', len(a_list))

number of words/phrases in the list: 305868


In [20]:
def chunk_word_into_letter_ngrams(word, n):
    ngrams = []
    for i in range(len(word)-n+1):
        ngrams.append( word[i : i+n] )
    return set(ngrams)

In [21]:
print (chunk_word_into_letter_ngrams('hello world', 3))

{'hel', 'rld', 'o w', 'orl', 'lo ', 'ell', ' wo', 'wor', 'llo'}


In [22]:
# You need a function that can calculate the Jaccard similarity for any pair of words

In [23]:
# You also need a function that calculates the edit distance for any pair of words
# (You can use an external package to calculate edit distance, e.g. the "editdistance" package)

In [24]:
# For each given string, you need to find a list of 10 correctly-spelled words from enwiktionary.a.list
#     that have the _highest_ n-gram Jaccard similarity to the given word
# Different lengths of the n-grams (i.e., different n) will likely produce a different list

In [25]:
# For each given string, you need to find a list of 10 correctly-spelled words from enwiktionary.a.list
#     that have the _lowest_ edit distance to the given word